In [1]:
!pip install scipy


# NUMPY SHAPE SUCKS 

In [2]:
import shapez2_render as Render 
import shapez2_gameinfo as GlobalInfos 
from shapez2_operation import *
from scipy.ndimage import convolve
from scipy.ndimage import label, generate_binary_structure



In [3]:

import numpy as np
from scipy.ndimage import convolve
from typing import List, Tuple


LAYERS_MAX = 4
LAYERS_QUAD_NUM = 4
LAYER_SEPARATOR_CHAR = ":"

VOID_SHAPE = "--"
PIN_SHAPE = "P-"
CRYSTAL_SHAPE = "cu"

SLIME_SHAPES = [VOID_SHAPE, PIN_SHAPE]
CRYSTALISED_SHAPES = [VOID_SHAPE, PIN_SHAPE]
CRYSTAL_SHAPES = ['c-', 'cu', 'cr']

SHAPE_NOTHING_CHAR = "-"
SHAPE_FULL_CHAR = "R"
SHAPE_PIN_CHAR = "P"
SHAPE_CRYSTAL_CHAR = "c"



In [4]:

def get_connected_components(matrix2D):
    # Label connected components
    structure = [[0, 1, 0], [1, 1, 1], [0, 1, 0] ]  # 8-connectivity
    structure = np.array(structure).astype(int)
    labeled_array, num_features = label(matrix2D, structure=structure)
    
    # Create a dictionary to store coordinates for each component
    components = []
    for component_id in range(1, num_features + 1):
        coordinates = np.argwhere(labeled_array == component_id)
        components.append(coordinates.tolist())
    return components

def get_connected_components_3D(matrix3D):
    # Label connected components
    structure = generate_binary_structure(3, 1)
     # connectivity
    structure = np.array(structure).astype(bool)
    labeled_array, num_features = label(matrix3D, structure=structure)
    
    # Create a dictionary to store coordinates for each component
    components = []
    for component_id in range(1, num_features + 1):
        coordinates = np.argwhere(labeled_array == component_id)
        components.append(coordinates.tolist())
    return components

In [151]:
def _breakCrystalCut(matrix):
    # Split the matrix along the x dimension
    matrix1 = matrix[:, :, :1]
    matrix2 = matrix[:, :, 1:]

    # mask cystal shape
    crystalShape = np.isin(matrix[:, :, :], CRYSTAL_SHAPES)

    # find connected group
    groupCrystal = get_connected_components_3D(crystalShape)

    # get list of crystal the have a connected side cut . ( only on hafl of the part)
    connectedCutCrystal = (matrix1 == matrix2) & (np.isin(matrix1, CRYSTAL_SHAPES))

    # get the coord of the break crystal
    cutCrystalCoords = np.argwhere(connectedCutCrystal)
    cutCrystalCoordsList = cutCrystalCoords.tolist()

    for breakPeace in cutCrystalCoordsList:
        # Find the group where a peace break
        breakGroup = next(
            (index for index, sublist in enumerate(groupCrystal) if breakPeace in sublist), 
            None
        )
        # Break the crystal group
        if (breakGroup is not None): 
            for z, x, y in groupCrystal[breakGroup]:
                matrix[z, x, y] = VOID_SHAPE
    return matrix




In [152]:

def _apply_gravity(matrix):
    """
    Apply gravity to a 3D matrix considering the conditions for falling.
    
    Args:
    matrix (np.ndarray): A 3D numpy array with pieces and NOTHING_CHAR.
    
    Returns:
    np.ndarray: The updated matrix with gravity applied.
    """
    depth, rows, cols = matrix.shape


    def getQuadFallCoordinates(matrix, z):
        isPeace = ~ np.isin(matrix[z, :, :], VOID_SHAPE)
        voidBellow = np.isin(matrix[z+1, :, :], VOID_SHAPE)

        notFallingSpace = isPeace & ~ voidBellow 

        # look for connected peace
        kernel = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0] ])
        # Check connected quad at distance 0
        a = notFallingSpace.astype(int)
        neighbor_sum = convolve(a, kernel, mode='constant', cval=0)
        neighbor_sum = neighbor_sum > 0
        notFallingSpace = neighbor_sum & isPeace

        # Check connected quad at distance 1 by probagation
        a = notFallingSpace.astype(int)
        neighbor_sum = convolve(a, kernel, mode='constant', cval=0)
        neighbor_sum = neighbor_sum > 0
        notFallingSpace = neighbor_sum & isPeace

        # Mask of Falling peace
        fallingPeace = ~ notFallingSpace & isPeace
        # print (" ----- fallingPeace \n", fallingPeace)

        coordinates = get_connected_components(fallingPeace)
        # Extract the coordinates of True values
        # coordinates = np.argwhere(fallingPeace)
        # print ( "fall peace", coordinates)
        return coordinates
    

    # Check the layer from the bottom to the top ( withou testing the first)
    for z in range(depth - 2, -1, -1):

        coordinates = getQuadFallCoordinates(matrix, z)
        for group in coordinates:
            for x, y in group:
                if (matrix[z, x, y] in CRYSTAL_SHAPES):
                    matrix[z, x, y] = VOID_SHAPE


        coordinates = getQuadFallCoordinates(matrix, z)
        # Move the item below 
        for group in coordinates:
            maxz = 4 - z
            for x, y in group:
                # Mask of empty space bellow
                isShapeBellow = ~ np.isin(matrix[z+1:, x, y], VOID_SHAPE)

                # Find the last index of void 
                # Reverse the array, find the first True (which is the last True in original array)
                last_true_reversed_index = np.argmax(isShapeBellow)
                # Compute the index in the original array
                last_index = last_true_reversed_index if np.any(isShapeBellow) else len(isShapeBellow)
                maxz = last_index if last_index and maxz > last_index else maxz
                # print ('index', last_index)

            for x, y in group:
                # Move the items
                matrix[z + maxz, x, y] = matrix[z, x, y]
                matrix[z, x, y] = VOID_SHAPE

        # print (adjacent_slime_mask)
        
    return matrix


In [153]:
# Find the first empty layer in the first matrix
def _first_empty_layer(matrix):
    for z in range(matrix.shape[0] - 1, -1, -1):
        if np.all(matrix[z] == VOID_SHAPE):
            return z
    return None

def _nlayerShape(matrix):
    idx_empty_layer = _first_empty_layer(matrix)
    if (idx_empty_layer is None): 
        return 4
    return 4 - idx_empty_layer - 1

In [221]:
class Shape:
    def __init__(self, matrix: np.ndarray) -> None:
        # Check if the list can be reshaped into a 2x2x4 array
        # if len(data_list) != 2 * 2 * 4:
        #     raise ValueError("The list must contain exactly 16 elements to fit into a 2x2x4 matrix.")
        
        # Convert the list to a NumPy array and reshape it to 2x2x4
        self.matrix:np.ndarray = matrix


    @classmethod
    def fromListOfLayers(cls, layers: list[str]) -> Self:
        num_layers = len(layers)
        num_quads = len(layers[0]) // 2
        data_list = [layer[i*2:i*2+2] for layer in layers for i in [1, 2, 0, 3]]
        if len(layers) < 4:
            nemptylayer = 4 - num_layers
            data_list = [*data_list , *(['--'] * nemptylayer * 4)]
        data_list.reverse()
        matrix: np.ndarray = np.array(data_list).reshape((4, 2, 2))
        return cls(matrix)

    @classmethod
    def fromShapeCode(cls, shapeCode: str) -> Self:
        return cls.fromListOfLayers(shapeCode.split(LAYER_SEPARATOR_CHAR))
    
    def toCode(self) -> str:
        depth = self.matrix.shape[0]
        s = ''
        for z in range(depth - 1, -1, -1):
            row = self.matrix[z]
            if np.all(row == VOID_SHAPE):
                break
            if z < depth - 1:
                s += ":"  # Separate layers with ':'
            s += row[0][1] + row[1][1] + row[1][0] + row[0][0]
        return s

    # def __repr__(self) -> str:
    #     return f"Shape(matrix={self.matrix})"
    def __repr__(self) -> str:
        # return f"Shape(matrix={self.matrix})"
        """Print the shape layers."""
        # self.layers
        layers = self.matrix # [    [s[i:i+2] for i in range(0, len(s), 2)] for s in self.layers]
        h1, h2 = "", ""
        s1 = ""
        s2 = ""
        for l in range(0, 4):
            pa, pb, pc, pd = layers[l][0][0], layers[l][0][1], layers[l][1][0], layers[l][1][1]
            h1 += "_____    "
            s1 += pa + " " + pb + "    "
            s2 += pc + " " + pd + "    "
            h2 += "⎺⎺⎺⎺⎺    "
        return '\n'.join([h1, s1, s2, h2])
    

def cut(shape:Shape) -> list[Shape]:
    matrix = shape.matrix.copy()

    matrix = _breakCrystalCut(matrix)
    # Split the matrix along the x dimension
    matrix1 = matrix[:, :, :1]
    matrix2 = matrix[:, :, 1:]

    # Pad the matrices to form (4, 2, 2) by adding zeros
    matrix1_padded = np.pad(matrix1, ((0, 0), (0, 0), (0, 1)), mode='constant', constant_values=VOID_SHAPE)
    matrix2_padded = np.pad(matrix2, ((0, 0), (0, 0), (1, 0)), mode='constant', constant_values=VOID_SHAPE)

    matrix1_padded = _apply_gravity(matrix1_padded)
    matrix2_padded = _apply_gravity(matrix2_padded)

    return Shape(matrix1_padded), Shape(matrix2_padded)

def rot90(shape:Shape) -> 'Shape':
    return  Shape(np.rot90(shape.matrix, -1, (1, 2)))

def rot180(shape:Shape) -> 'Shape':
    return  Shape(np.rot90(shape.matrix, -2, (1, 2)))
def rot270(shape:Shape) -> 'Shape':
    return  Shape(np.rot90(shape.matrix, -3, (1, 2)))

# def cut(shape:Shape) -> list[Shape]:
#     matrix = shape.matrix
#     return Shape(shape)




In [222]:
shape2 = Shape.fromShapeCode('--Su--Su:A-B-C-D-')
print (shape2)
print (rot180(shape2))

_____    _____    _____    _____    
-- --    -- --    D- A-    Su --    
-- --    -- --    C- B-    -- Su    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    
_____    _____    _____    _____    
-- --    -- --    B- C-    Su --    
-- --    -- --    A- D-    -- Su    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    


In [314]:

def swapHalves(shapeA:Shape,shapeB:Shape) -> list[Shape]:

    matrix1 = _breakCrystalCut(shapeA.matrix.copy())
    matrix2 = _breakCrystalCut(shapeB.matrix.copy())

    # Perform the swap
    new_matrix1 = matrix1.copy()
    new_matrix2 = matrix2.copy()

    slice1 = slice(0, 1)  # First half
    slice2 = slice(1, 2)  # Second half

    new_matrix1[:, :, slice1] = matrix2[:, :, slice1]
    new_matrix1[:, :, slice2] = matrix1[:, :, slice2]

    new_matrix2[:, :, slice1] = matrix1[:, :, slice1]
    new_matrix2[:, :, slice2] = matrix2[:, :, slice2]
    new_matrix1 = _apply_gravity(new_matrix1)
    new_matrix2 = _apply_gravity(new_matrix2)
    return Shape(new_matrix1), Shape(new_matrix2)

def stack(shapeA:Shape,shapeB:Shape) -> Shape:

    # TODO : Break Crystal on the top . 

    empty_layer_index = _first_empty_layer(shapeA.matrix)
    if empty_layer_index is None:
        # Shape is full 
        return shapeA

    # Create a new matrix with the same shape as matrix1
    new_matrix = shapeA.matrix.copy()
    
    # Break the chistal on the top layer
    m2 = shapeB.matrix.copy()
    m2_crystal_mask = np.isin(m2, CRYSTAL_SHAPES)
    m2[m2_crystal_mask] = VOID_SHAPE

    # Swap a slice of bottom B , to the empty space at the top of A 
    new_matrix[slice(0, empty_layer_index + 1 ), : , : ] = m2[slice(4 - empty_layer_index - 1, 4), : , : ]
    new_matrix = _apply_gravity(new_matrix)

    return  Shape(new_matrix)

def pushPin(shape:Shape) -> Shape:

    def _topLvlCrystalGroup(matrix):
        # mask cystal shape
        crystalShape = np.isin(matrix[:, :, :], CRYSTAL_SHAPES)

        # find connected group
        groupCrystal = get_connected_components_3D(crystalShape)
        # get list of crystal on the top of the shape 
        connectedCutCrystal = np.isin(matrix, CRYSTAL_SHAPES) 
        connectedCutCrystal[slice(1 , 4), :, :] = False
        # get the coord of the break crystal
        cutCrystalCoords = np.argwhere(connectedCutCrystal)
        breakCrystalCoordsList = cutCrystalCoords.tolist()
        return breakCrystalCoordsList, groupCrystal

    def _breakCrystalUp(matrix, breakCordList, groupCrystal):
        for breakPeace in breakCordList:
            # Find the group where a peace break
            breakGroupIdx = next(
                (index for index, sublist in enumerate(groupCrystal) if breakPeace in sublist), 
                None
            )
            # Break the crystal group
            if (breakGroupIdx is not None): 
                for z, x, y in groupCrystal[breakGroupIdx]:
                    # Break AfterPin -> Offset
                    if z > 0:
                        matrix[z - 1, x, y] = VOID_SHAPE
        return matrix

    matrix = shape.matrix.copy()

    # Check if exist a crystal on the top 
    breakCrystalCoordsList, groupCrystal = _topLvlCrystalGroup(matrix)

    # Create a full pin shape
    new_matrix = np.array([PIN_SHAPE] * 4 * 4).reshape(4, 2, 2)
    # Swap the 3 layer of the shape on top 
    new_matrix[slice(0, 3), :, :] = matrix[slice(1, 4), :, :] 

    # Clean the pin is void above
    empty_above = new_matrix[2] == VOID_SHAPE
    new_matrix[3][empty_above] = VOID_SHAPE

    # Print the Crystal 
    if breakCrystalCoordsList:
        _breakCrystalUp(new_matrix, breakCrystalCoordsList, groupCrystal)
        
    new_matrix = _apply_gravity(new_matrix)

    return Shape(new_matrix)

def genCrystal(shape:Shape) -> Shape:
    matrix = shape.matrix.copy()
    # Find the first empty layer in the first matrix
    empty_layer_index = _first_empty_layer(matrix) 
    empty_layer_index = empty_layer_index + 1 if empty_layer_index is not None else 0

    crystalMask = np.isin(matrix, CRYSTALISED_SHAPES)
    crystalMask[slice(0 , empty_layer_index), :, :] = False
    
    matrix[crystalMask] = CRYSTAL_SHAPE

    return Shape(matrix)

In [315]:
if False:
    # Error: genCrystal:  P-P-P-P-:cucucucu:cucucucu:cucuCucu
    # ref: cucucucu:cucucucu:cucucucu:cucuCucu
    # new: P-P-P-P-:cucucucu:cucucucu:cucuCucu
    shape1 = Shape.fromShapeCode('P-P-P-P-:cucucucu:cucucucu:cucuCucu')
    ca = genCrystal(shape1)
    print (ca)
    print (ca.toCode())
    print ('=' * 80)

if False:
    """
    Error: pushPin:  P-P-P-P-:P-P-P-P-:cucucucu:CucuCuCu
    ref: P-P-P-P-:P-P-P-P-:P-P-P-P-
    new: P-P-P-P-:P-P-P-P-:P-P-P-P-:cucucucu
    """
    shape1 = Shape.fromShapeCode('P-P-P-P-:P-P-P-P-:cucucucu:CucuCuCu')
    ca = pushPin(shape1)
    print (ca)
    print (ca.toCode())
    print ('=' * 80)


if False:
    """
    Error: pushPin:  cucucucu:cucucucu:cucucucu:cucuCucu
    ref: P-P-P-P-
    new: 
    """
    shape1 = Shape.fromShapeCode('cucucucu:cucucucu:cucucucu:cucuCucu')
    ca = pushPin(shape1)
    print (ca)
    print (ca.toCode())
    print ('=' * 80)

if True:
    """
    ref: --CuCuCu
    new: ----CuCu:--Cu----
    Error: Swap A:  ----Cu--:--CuCu-- CuCuCuCu
    """
    shape1 = Shape.fromShapeCode('----Cr--:--CrCr--')
    shape2 = Shape.fromShapeCode('CuCuCuCu')
    ca , cb = swapHalves(shape1, shape2)
    print (shape1)
    print (shape2)
    print ('-' * 100)
    print (ca)
    print (cb)
    print (ca.toCode())
    print (cb.toCode())
    print ('=' * 80)



_____    _____    _____    _____    
-- --    -- --    -- --    -- --    
-- --    -- --    Cr Cr    Cr --    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    
_____    _____    _____    _____    
-- --    -- --    -- --    Cu Cu    
-- --    -- --    -- --    Cu Cu    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    
----------------------------------------------------------------------------------------------------
_____    _____    _____    _____    
-- --    -- --    -- --    Cu --    
-- --    -- --    -- --    Cu Cr    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    
_____    _____    _____    _____    
-- --    -- --    -- --    -- Cu    
-- --    -- --    Cr --    Cr Cu    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    
--CrCuCu
CuCuCr--:----Cr--


In [316]:
"""
def swapHalves(shapeA:Shape,shapeB:Shape) -> list[Shape]:
    numLayers = max(shapeA.numLayers,shapeB.numLayers)
    takeQuads = math.ceil(shapeA.numQuads/2)
    shapeACut, shapeBCut = coreCut(shapeA), coreCut(shapeB)
    shapeACut = [[*s.layers,*([[Quadrant(NOTHING_CHAR,NOTHING_CHAR)]*shapeA.numQuads]*(numLayers-len(s.layers)))] for s in shapeACut]
    shapeBCut = [[*s.layers,*([[Quadrant(NOTHING_CHAR,NOTHING_CHAR)]*shapeB.numQuads]*(numLayers-len(s.layers)))] for s in shapeBCut]
    returnShapeA = []
    returnShapeB = []
    for layerA0,layerA1,layerB0,layerB1 in zip(*shapeACut,*shapeBCut):
        returnShapeA.append([*(layerA1[:-takeQuads]),*(layerB0[-takeQuads:])])
        returnShapeB.append([*(layerB1[:-takeQuads]),*(layerA0[-takeQuads:])])
    returnShapeA, returnShapeB = _cleanUpEmptyUpperLayers(returnShapeA),_cleanUpEmptyUpperLayers(returnShapeB)
    return [Shape(returnShapeA),Shape(returnShapeB)]

def coreStack(bottomShape:Shape,topShape:Shape) -> list[Shape]:
    newTopShape = [[Quadrant(NOTHING_CHAR,NOTHING_CHAR) if q.shape == CRYSTAL_CHAR else q for q in l] for l in topShape.layers]
    newLayers = [*bottomShape.layers,*newTopShape]
    newLayers = _cleanUpEmptyUpperLayers(_makeLayersFall(newLayers))
    newLayers = newLayers[:MAX_SHAPE_LAYERS]
    return [Shape(newLayers)]

def topPaint(shape:Shape,color:str) -> list[Shape]:
    newLayers = shape.layers[:-1]
    newLayers.append([Quadrant(q.shape,q.color if q.shape in UNPAINTABLE_SHAPES else color) for q in shape.layers[-1]])
    return [Shape(newLayers)]

def pushPin(shape:Shape) -> list[Shape]:

    layers = shape.layers
    addedPins = []

    for quad in layers[0]:
        if quad.shape == NOTHING_CHAR:
            addedPins.append(Quadrant(NOTHING_CHAR,NOTHING_CHAR))
        else:
            addedPins.append(Quadrant(PIN_CHAR,NOTHING_CHAR))

    if len(layers) < MAX_SHAPE_LAYERS:
        newLayers = [addedPins,*layers]
    else:
        newLayers = [addedPins,*(layers[:MAX_SHAPE_LAYERS-1])]
        removedLayer = layers[MAX_SHAPE_LAYERS-1]
        for quadIndex,quad in enumerate(newLayers[MAX_SHAPE_LAYERS-1]):
            if _crystalsFused(quad,removedLayer[quadIndex]):
                _breakCrystals(newLayers,MAX_SHAPE_LAYERS-1,quadIndex)
        newLayers = _cleanUpEmptyUpperLayers(_makeLayersFall(newLayers))

    return [Shape(newLayers)]

def genCrystal(shape:Shape,color:str) -> list[Shape]:
    return [Shape([[Quadrant(CRYSTAL_CHAR,color) if q.shape in REPLACED_BY_CRYSTAL else q for q in l] for l in shape.layers])]

""" 

'\ndef swapHalves(shapeA:Shape,shapeB:Shape) -> list[Shape]:\n    numLayers = max(shapeA.numLayers,shapeB.numLayers)\n    takeQuads = math.ceil(shapeA.numQuads/2)\n    shapeACut, shapeBCut = coreCut(shapeA), coreCut(shapeB)\n    shapeACut = [[*s.layers,*([[Quadrant(NOTHING_CHAR,NOTHING_CHAR)]*shapeA.numQuads]*(numLayers-len(s.layers)))] for s in shapeACut]\n    shapeBCut = [[*s.layers,*([[Quadrant(NOTHING_CHAR,NOTHING_CHAR)]*shapeB.numQuads]*(numLayers-len(s.layers)))] for s in shapeBCut]\n    returnShapeA = []\n    returnShapeB = []\n    for layerA0,layerA1,layerB0,layerB1 in zip(*shapeACut,*shapeBCut):\n        returnShapeA.append([*(layerA1[:-takeQuads]),*(layerB0[-takeQuads:])])\n        returnShapeB.append([*(layerB1[:-takeQuads]),*(layerA0[-takeQuads:])])\n    returnShapeA, returnShapeB = _cleanUpEmptyUpperLayers(returnShapeA),_cleanUpEmptyUpperLayers(returnShapeB)\n    return [Shape(returnShapeA),Shape(returnShapeB)]\n\ndef coreStack(bottomShape:Shape,topShape:Shape) -> list[Sha

In [317]:
## shape = Shape.fromShapeCode('Ru--Ru--:--Cu--Cu')
## shape = Shape.fromShapeCode('Ru--Ru--:RuRuRu--:Ru--Ru--:--Cu--Cu')

# shape = Shape.fromShapeCode('Ru----Ru:Cu------:----WuWu:--Su--Su')
# shape = Shape.fromShapeCode('Ru----Ru:Cu------:----WuWu')
# shape = Shape.fromShapeCode('Ru------:Cu--cr--:--------')
# shape = Shape.fromShapeCode('CuCucrcr:------cr')
# shape = Shape.fromShapeCode('crCuCucr:cr------')

# shape1 = Shape.fromShapeCode('Ru----Ru:RuRuRuRu')
shape3 = Shape.fromShapeCode('Cu------')

shape1 = Shape.fromShapeCode('RuP-P-Ru:Cu------:----WuWu')
shape2 = Shape.fromShapeCode('--Su--Su:A-B-C-D-')


# rotated_shape = shape.rotate_z_90()
# print (rotated_shape)
# shape = shape.apply_gravity_with_conditions()
# print (shape)

# ca , cb = cut(shape)
# cc = rot90(ca)
# ca  = stack(shape1, shape2)

ca = genCrystal(shape1)
print (ca)
print (ca.toCode())
print ('=' * 80)
# print (ca.toCode())
# print (ca.toCode())

# nlayerShape(shape3.matrix)


_____    _____    _____    _____    
-- --    Wu cu    cu Cu    Ru Ru    
-- --    Wu cu    cu cu    cu cu    
⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    ⎺⎺⎺⎺⎺    
RucucuRu:Cucucucu:cucuWuWu


# OLD REF 

In [318]:
import shapez2_render as Render 
import shapez2_gameinfo as GlobalInfos 
import shapez2_operation as ref


In [361]:
class RefShape:
    def __init__(self, shapeCode:str) -> None:
        self.shape = ref.Shape.fromShapeCode(shapeCode)
  
    def toCode(self) -> str:
        return self.shape.toShapeCode()

    def __repr__(self) -> str:
        
        layers = self.shape.toListOfLayers()
        layers = [    [s[i:i+2] for i in range(0, len(s), 2)] for s in layers]
        h1, h2 = "", ""
        s1 = ""
        s2 = ""
        for l in range(0, len(layers)):
            h1 += "_____    "
            s1 += layers[l][3] + " " + layers[l][0] + "    "
            s2 += layers[l][2] + " " + layers[l][1] + "    "
            h2 += "⎺⎺⎺⎺⎺    "
        return ('\n'.join([h1, s1, s2, h2]))
        # for layer
        # print(layers)
        # layers.reverse()
        # print ("________\n" + '\n'.join(layers) + "\n⎺⎺⎺⎺⎺⎺⎺⎺")

              
def refrot90(s:RefShape) -> RefShape:
    shape2 = ref.rotate90CW(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refrot270(s:RefShape) -> RefShape:
    shape2 = ref.rotate90CCW(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refrot180(s:RefShape) -> RefShape:
    shape2 = ref.rotate180(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refpushpin(s:RefShape) -> RefShape:
    shape2 = ref.pushPin(s.shape)[0]
    return RefShape(shape2.toShapeCode())

def refgenCrystal(s:RefShape) -> RefShape:
    shape2 = ref.genCrystal(s.shape, "u")[0]
    return RefShape(shape2.toShapeCode())


def refcut(s:RefShape) -> list[Self]:
    shape = ref.coreCut(s.shape)
    return RefShape(shape[0].toShapeCode()), RefShape(shape[1].toShapeCode())

def refswap(s:RefShape, shapeB:RefShape) -> list[RefShape]:
    shape = ref.swapHalves(s.shape, shapeB.shape)
    return RefShape(shape[0].toShapeCode()), RefShape(shape[1].toShapeCode())

def refstack(s:RefShape, topShape:RefShape) -> RefShape:
    shape = ref.coreStack(s.shape, topShape.shape)
    return RefShape(shape[0].toShapeCode())



In [366]:
refa = RefShape('CuCuCuCu:Cu--Cu--')

a = Shape.fromShapeCode('CuCuCuCu:Cu--Cu--')

# print (a)

for it in range (0, 10000):
    # a = 1 + 1 
    # fa, fb = cut(a)
    # print(fa)

    rfa, rfb = refcut(refa)
    # print (rfa)

# INDEX 

In [367]:
# INDEX

dataAll = {}
keyCheck = []
keyUncheck = []
keyCheckLayerMax = []

# Function to check if a key exists
def key_exists(key: str) -> bool:
    return key in dataAll

def get_keys() -> bool:
    return [k for k, v in dataAll]

# Function to add or update a key/value pair
def add_key_value(key: str, nlayer:int, op:str, parentA: str="", rotationA:int=0, parentB: str="", rotationB:int=0, opindex:int=0) -> None:
    dataAll[key] = [False, nlayer, op, parentA, rotationA, parentB, rotationB, opindex]
    keyUncheck.append(key)

# Function to retrieve a value by key
def get_value(key: str) -> str:
    return dataAll[key] if key in dataAll else None

# Function to set "ischeck" for an existing key
def set_ischeck(key: str, ischeck_value: bool=True) -> None:
    if (dataAll[key]):
        keyCheck.append(key)
        if (dataAll[key][1] == 1):
            keyCheckLayerMax.append(key)
        
# Function to get the first key with ischeck = False
def get_first_unchecked_key() -> str:
    return keyUncheck.pop() if keyUncheck else None

# Function to get all the checked keys
def get_checked_key() -> str:
    return keyCheckLayerMax


def search_exact_shape(key:str) -> dict:
    if not key in dataAll : 
        return {}
    d = dataAll[key]
    nlayer, operator, opindex, parentA, rotationA, parentB, rotationB, ischeck = d
    rt = {
        'shapeCode': key, 
        'op': operator, 
        'nlayer':nlayer,
        'parentA':parentA, 'rotationA': rotationA,
        'parentB':parentB, 'rotationB': rotationB,
        'ischeck':ischeck, 
        'opindex':opindex
    }
    return rt

# Function to get all the checked keys
def print_db_count() -> str:
    # Print the counts
    print(f"{'Total Checked':<30}{len(keyCheck):<10}")
    print(f"{'Total Unchecked':<30}{len(keyUncheck):<10}")
    return ""


In [321]:
add_key_value('CuCuCuCu', 1, 'start', '', 0, '', 0, 0)

# LOOP TEST REF 

In [379]:
def index_shape(mshape:RefShape, op:str, parentA, rotA:int, parentB: str, rotB:int, opindex:int=0) -> None:
    a = mshape
    b = refrot90(a)
    c = refrot90(b)
    d = refrot90(c)

    exit_key = any([key_exists(it.toCode()) for it in [a, b , c, d]])

    if (not exit_key):
        nlayers = mshape.shape.numLayers
        add_key_value(mshape.toCode(), nlayers, op, parentA, rotA, parentB, rotB, opindex)


In [380]:

def _test_pin(mshape:RefShape):
    shapeCode = mshape.toCode()
    a = refpushpin(mshape)
    index_shape(a, 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   

def _test_genCrytal(mshape:RefShape):
    shapeCode = mshape.toCode()
    a = refgenCrystal(mshape)
    index_shape(a, 'crys', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   


def _test_cut(mshape:RefShape):
    shapeCode = mshape.toCode()
    a, b = refcut(mshape)
    index_shape(a, 'cut', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   
    index_shape(b, 'cut', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=1)   


def _test_swap(mshapeA:RefShape, mshapeB: RefShape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a, b = refswap(mshapeA, mshapeB)
    index_shape(a, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   
    index_shape(b, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=1)   


def _test_stak(mshapeA:RefShape, mshapeB: RefShape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a = refstack(mshapeA, mshapeB)
    index_shape(a, 'stack', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   


In [383]:


def apply_shape_function():
    shapeCode = get_first_unchecked_key()
    if (shapeCode == '--------'): return True
    if (not shapeCode):
        print ('####################### UncheckCode Not Found ##################################### ')
        return False

    mshape = RefShape(shapeCode)
    # index_shape(mshape.rotC(), 'rot', shapeCode, '', 0)   
    
    _test_pin(mshape)
    # index_shape(pushPin(mshape), 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   
    
    _test_genCrytal(mshape)
    
    _test_cut(mshape)
    _test_cut(refrot90(mshape))

    # nlayer = _nlayerShape(mshape.matrix)
    nlayer = mshape.shape.numLayers
    if (nlayer > 2): 
        set_ischeck(shapeCode)
        return True

    for it in keyCheckLayerMax:
        shapeCodeB = it
        mshapeB = RefShape(shapeCodeB)

        _test_swap(mshape, mshapeB)
        _test_swap(mshape, refrot90(mshapeB))
        _test_swap(refrot90(mshape), refrot90(mshapeB))
        _test_swap(refrot90(mshape), mshapeB)

        _test_stak(mshape, mshapeB)
        _test_stak(mshape, refrot90(mshapeB))
        _test_stak(refrot90(mshape), refrot90(mshapeB))
        _test_stak(refrot90(mshape), mshapeB)

        _test_stak(mshapeB, mshape)
        _test_stak(mshapeB, refrot90(mshape))
        _test_stak(refrot90(mshapeB), refrot90(mshape))
        _test_stak(refrot90(mshapeB), mshape)


    set_ischeck(shapeCode)
    return True


In [384]:
from IPython.display import clear_output
import time


dataAll = {}
keyCheck = []
keyUncheck = []
keyCheckLayerMax = []

add_key_value('CuCuCuCu', 1, 'start', '', 0, '', 0, 0)

i = 0
noFinish = True
while noFinish:
    i += 1
    isapply = apply_shape_function()
    if (not isapply):
        noFinish = False
    if (i % 100 == 0):
        # clear_output(wait=True)
        print('---------------------------', i)
        print_db_count()

# print_db(10)

print ('SIZE' , len(dataAll))

--------------------------- 100
Total Checked                 100       
Total Unchecked               263       
--------------------------- 200
Total Checked                 200       
Total Unchecked               348       
--------------------------- 300
Total Checked                 300       
Total Unchecked               432       
--------------------------- 400
Total Checked                 400       
Total Unchecked               434       
--------------------------- 500
Total Checked                 500       
Total Unchecked               439       
--------------------------- 600
Total Checked                 600       
Total Unchecked               548       
--------------------------- 700
Total Checked                 700       
Total Unchecked               552       
--------------------------- 800
Total Checked                 800       
Total Unchecked               648       
--------------------------- 900
Total Checked                 900       
Total Unchecked

In [385]:
dataAll

{'CuCuCuCu': [False, 1, 'start', '', 0, '', 0, 0],
 'P-P-P-P-:CuCuCuCu': [False, 2, 'pin', 'CuCuCuCu', 0, '', 0, 0],
 '----CuCu': [False,
  1,
  'cut',
  'CuCuCuCu',
  <function shapez2_operation.rot(shape: shapez2_operation.Shape) -> list>,
  '',
  0,
  0],
 '----P-P-:----CuCu': [False, 2, 'pin', '----CuCu', 0, '', 0, 0],
 'cucuCuCu': [False, 1, 'crys', '----CuCu', 0, '', 0, 0],
 '--------': [False,
  1,
  'cut',
  '----CuCu',
  <function shapez2_operation.rot(shape: shapez2_operation.Shape) -> list>,
  '',
  0,
  1],
 '------Cu': [False,
  1,
  'cut',
  'Cu----Cu',
  <function shapez2_operation.rot(shape: shapez2_operation.Shape) -> list>,
  '',
  0,
  0],
 'Cu--CuCu': [False, 1, 'swap', 'Cu----Cu', 0, 'CuCuCuCu', 0, 0],
 '----CuCu:CuCuCuCu': [False, 2, 'stack', '----CuCu', 0, 'CuCuCuCu', 0, 0],
 'CuCuCuCu:----CuCu': [False, 2, 'stack', 'CuCuCuCu', 0, '----CuCu', 0, 0],
 'P-P-P-P-:CuCuCuCu:----CuCu': [False,
  3,
  'pin',
  'CuCuCuCu:----CuCu',
  0,
  '',
  0,
  0],
 'CuCuCuCu:cucuCu

# LOOP TO TEST 

In [337]:
def index_shape(mshape:Shape, op:str, parentA, rotA:int, parentB: str, rotB:int, opindex:int=0) -> None:
    a = mshape
    b = rot90(a)
    c = rot90(b)
    d = rot90(c)

    exit_key = any([key_exists(it.toCode()) for it in [a, b , c, d]])

    if (not exit_key):
        nlayers = _nlayerShape(mshape.matrix) 
        add_key_value(mshape.toCode(), nlayers, op, parentA, rotA, parentB, rotB, opindex)


In [338]:
def _errorShape(mshape:Shape, refshape:RefShape):
    mcode = mshape.toCode()
    refcode = refshape.toCode()
    def _clean(code):
        # code = '' if mcode == '--------' else code
        return code

    if _clean(mcode) != _clean(refcode):
        print ('-'* 30)
        print ("ref:", refcode)
        print ("new:", mcode)
        return True
    return False



def _test_pin(mshape:Shape):
    shapeCode = mshape.toCode()
    a = pushPin(mshape)
    index_shape(a, 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   

    refa = refpushpin( RefShape(shapeCode))
    if _errorShape(a, refa):
        print ("Error: pushPin: ", mshape.toCode())


def _test_genCrytal(mshape:Shape):
    shapeCode = mshape.toCode()
    a = genCrystal(mshape)
    index_shape(a, 'crys', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   

    refa = refgenCrystal(RefShape(shapeCode))
    if _errorShape(a, refa):
        print ("Error: genCrystal: ",shapeCode)



def _test_cut(mshape:Shape):
    shapeCode = mshape.toCode()
    a, b = cut(mshape)
    index_shape(a, 'cut', parentA=shapeCode, rotA=rot, parentB='', rotB=0, opindex=0)   
    index_shape(b, 'cut', parentA=shapeCode, rotA=rot, parentB='', rotB=0, opindex=1)   

    refa , refb = refcut(RefShape(mshape.toCode()))
    if _errorShape(a, refa):
        print (f"Error: cut A  ", shapeCode)
    if _errorShape(b, refb):
        print (f"Error: cut B  ", shapeCode)



def _test_swap(mshapeA:Shape, mshapeB: Shape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a, b = swapHalves(mshapeA, mshapeB)
    index_shape(a, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   
    index_shape(b, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=1)   


    refa , refb = refswap( RefShape(shapeCodeA), RefShape(shapeCodeB)) 
    if _errorShape(a, refa):
        print (f"Error: Swap A: ", shapeCodeA, shapeCodeB)
    if _errorShape(b, refb):
        print (f"Error: Swap B: ", shapeCodeA, shapeCodeB)


def _test_stak(mshapeA:Shape, mshapeB: Shape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a = stack(mshapeA, mshapeB)
    index_shape(a, 'stack', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   

    refa = refstack( RefShape(shapeCodeA), RefShape(shapeCodeB)) 
    if _errorShape(a, refa):
        print (f"Error: Stack A: ", shapeCodeA, shapeCodeB)

    

In [344]:

def _test_pin(mshape:Shape):
    shapeCode = mshape.toCode()
    a = pushPin(mshape)
    index_shape(a, 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   

def _test_genCrytal(mshape:Shape):
    shapeCode = mshape.toCode()
    a = genCrystal(mshape)
    index_shape(a, 'crys', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   


def _test_cut(mshape:Shape):
    shapeCode = mshape.toCode()
    a, b = cut(mshape)
    index_shape(a, 'cut', parentA=shapeCode, rotA=rot, parentB='', rotB=0, opindex=0)   
    index_shape(b, 'cut', parentA=shapeCode, rotA=rot, parentB='', rotB=0, opindex=1)   


def _test_swap(mshapeA:Shape, mshapeB: Shape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a, b = swapHalves(mshapeA, mshapeB)
    index_shape(a, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   
    index_shape(b, 'swap', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=1)   


def _test_stak(mshapeA:Shape, mshapeB: Shape):
    shapeCodeA = mshapeA.toCode()
    shapeCodeB = mshapeB.toCode()

    a = stack(mshapeA, mshapeB)
    index_shape(a, 'stack', parentA=shapeCodeA, rotA=0, parentB=shapeCodeB, rotB=0, opindex=0)   

    

In [349]:


def apply_shape_function():
    shapeCode = get_first_unchecked_key()
    if (shapeCode == '--------'): return True
    if (not shapeCode):
        print ('####################### UncheckCode Not Found ##################################### ')
        return False

    mshape = Shape.fromShapeCode(shapeCode)
    # index_shape(mshape.rotC(), 'rot', shapeCode, '', 0)   
    
    _test_pin(mshape)
    # index_shape(pushPin(mshape), 'pin', parentA=shapeCode, rotA=0, parentB='', rotB=0, opindex=0)   
    
    _test_genCrytal(mshape)
    
    _test_cut(mshape)
    _test_cut(rot90(mshape))

    nlayer = _nlayerShape(mshape.matrix)
    if (nlayer > 4): 
        set_ischeck(shapeCode)
        return True

    for it in keyCheckLayerMax:
        shapeCodeB = it
        mshapeB = Shape.fromShapeCode(shapeCodeB)

        _test_swap(mshape, mshapeB)
        _test_swap(mshape, rot90(mshapeB))
        _test_swap(rot90(mshape), rot90(mshapeB))
        _test_swap(rot90(mshape), mshapeB)

        _test_stak(mshape, mshapeB)
        _test_stak(mshape, rot90(mshapeB))
        _test_stak(rot90(mshape), rot90(mshapeB))
        _test_stak(rot90(mshape), mshapeB)

        _test_stak(mshapeB, mshape)
        _test_stak(mshapeB, rot90(mshape))
        _test_stak(rot90(mshapeB), rot90(mshape))
        _test_stak(rot90(mshapeB), mshape)


    set_ischeck(shapeCode)
    return True



In [350]:
from IPython.display import clear_output
import time


dataAll = {}
keyCheck = []
keyUncheck = []
keyCheckLayerMax = []

add_key_value('CuCuCuCu', 1, 'start', '', 0, '', 0, 0)

i = 0
noFinish = True
while noFinish:
    i += 1
    isapply = apply_shape_function()
    if (not isapply):
        noFinish = False
    if (i % 100 == 0):
        # clear_output(wait=True)
        print('---------------------------', i)
        print_db_count()

# print_db(10)

print ('SIZE' , len(dataAll))

--------------------------- 100
Total Checked                 100       
Total Unchecked               864       
--------------------------- 200
Total Checked                 200       
Total Unchecked               1503      
--------------------------- 300
Total Checked                 300       
Total Unchecked               2044      
--------------------------- 400
Total Checked                 400       
Total Unchecked               2519      
--------------------------- 500
Total Checked                 500       
Total Unchecked               2923      
--------------------------- 600
Total Checked                 600       
Total Unchecked               3325      
--------------------------- 700
Total Checked                 700       
Total Unchecked               3780      
--------------------------- 800
Total Checked                 800       
Total Unchecked               4156      
--------------------------- 900
Total Checked                 900       
Total Unchecked

KeyboardInterrupt: 

In [332]:
len(dataAll)

3109

In [336]:
[key for key in dataAll]

['CuCuCuCu',
 'P-P-P-P-:CuCuCuCu',
 '----CuCu',
 '----P-P-:----CuCu',
 'cucuCuCu',
 '--------',
 '------Cu',
 'Cu--CuCu',
 '----CuCu:CuCuCuCu',
 'CuCuCuCu:----CuCu',
 'P-P-P-P-:CuCuCuCu:----CuCu',
 'CuCuCuCu:cucuCuCu',
 '----CuCu:----CuCu',
 '----CuCu:------Cu',
 'CuCu----:Cu------',
 'CuCuCuCu:Cu------',
 'CuCuCuCu:----CuCu:CuCuCuCu',
 'CuCuCuCu:CuCuCuCu:----CuCu',
 'P-P-P-P-:CuCuCuCu:CuCuCuCu:----CuCu',
 'CuCuCuCu:CuCuCuCu:cucuCuCu',
 '----CuCu:----CuCu:----CuCu',
 '----CuCu:----CuCu:------Cu',
 'CuCu----:CuCu----:Cu------',
 'CuCuCuCu:----CuCu:----CuCu',
 'CuCuCuCu:CuCu----:Cu------',
 'CuCuCuCu:----CuCu:------Cu',
 'CuCuCuCu:CuCuCuCu:----CuCu:CuCuCuCu',
 'CuCuCuCu:CuCuCuCu:CuCuCuCu:----CuCu',
 'P-P-P-P-:CuCuCuCu:CuCuCuCu:CuCuCuCu',
 'CuCuCuCu:CuCuCuCu:CuCuCuCu:cucuCuCu',
 '----CuCu:----CuCu:----CuCu:----CuCu',
 '----CuCu:----CuCu:----CuCu:------Cu',
 'CuCu----:CuCu----:CuCu----:Cu------',
 'P-P-----:CuCu----:CuCu----:CuCu----',
 'CuCucucu:CuCucucu:CuCucucu:Cucucucu',
 '----Cu--:---